# Insights Report

In [1]:
import facebookinsights as fi
import pandas as pd
import facebook

def generateCSVs(token, since, until):  
    '''
    Reactions: Like, love, wow, haha, sad, anger
    Impressions: Organic, organic unique viral
    Negative Feedback
    Positive Feedback
    Page Consumptions By Consumption Type
    Page fans online (average)
    Demographics
        Page fans and page fans by age/gender (average)
        Page fans by country
        Page likes by source
        Page unlikes by source
    '''
    # Auth
    page = fi.authenticate(token=token)
    graph = facebook.GraphAPI(access_token=token, version='2.7')

    # Requests
    reactions = graph.get_object(id="tecdemonterrey/insights",metric='page_actions_post_reactions_like_total,page_actions_post_reactions_love_total,page_actions_post_reactions_wow_total,page_actions_post_reactions_haha_total,page_actions_post_reactions_sorry_total,page_actions_post_reactions_anger_total',until=until,since=since)
    insights = page.insights.daily(['page_impressions_organic','page_impressions_organic_unique','page_impressions_viral','page_consumptions_by_consumption_type','page_negative_feedback_by_type','page_positive_feedback_by_type','page_fans_online']).range(months=1,since=since).get()
    countryCodes = pd.read_html('https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2')
    fansbyCountry = graph.get_object(id="tecdemonterrey/insights",metric='page_fans_country',until=until,since=since)
    unlikes = graph.get_object(id="tecdemonterrey/insights",metric='page_fans_by_unlike_source_unique',until=until,since=since)
    likes = graph.get_object(id="tecdemonterrey/insights",metric='page_fans_by_like_source_unique',until=until,since=since)
    fans = graph.get_object(id="tecdemonterrey/insights",metric='page_fans',until=until,since=since)
    fansByAge = graph.get_object(id="tecdemonterrey/insights",metric='page_fans_gender_age',until=until,since=since)

    # DataFrame creation
    impressionsdf = pd.DataFrame(columns=["Page Impressions Organic", "Page Impressions Organic Unique", "Page Impressions Viral"])
    negativeFeedbackdf = pd.DataFrame(columns=['hide_clicks','hide_all_clicks','report_spam_clicks','unlike_page_clicks','xbutton_clicks'])
    positiveFeedbackdf = pd.DataFrame(columns=['like','link','comment','answer','claim'])
    pageFansOnlinedf = pd.DataFrame(columns=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22','23'])
    onlineFansaveragedf = pd.DataFrame(columns=["Average"])
    pageConsumptionsdf = pd.DataFrame(columns=['other clicks','video play','photo view','link clicks'])
    reactionsdf = pd.DataFrame()
    pageUnlikesBySourcedf = pd.DataFrame()
    pageLikesBySourcedf = pd.DataFrame()
    fansDemographicsdf = pd.DataFrame()
    fansbyAgedf = pd.DataFrame()
    newdf = pd.DataFrame(columns=["13-17","18-24","25-34","35-44","45-54","55-64","65+"])

    # Data processing
    dates = []
    for i,insight in enumerate(insights):
        dates.append(insight.end_time.strftime("%d"))

        impressionsdf.loc[i,"Page Impressions Organic"] = insight.page_impressions_organic
        impressionsdf.loc[i,"Page Impressions Organic Unique"] = insight.page_impressions_organic_unique
        impressionsdf.loc[i,"Page Impressions Viral"] = insight.page_impressions_viral

        for column in pageConsumptionsdf:
            pageConsumptionsdf.loc[i,column] = insight.page_consumptions_by_consumption_type[column]

        for column in negativeFeedbackdf:
            negativeFeedbackdf.loc[i,column] = insight.page_negative_feedback_by_type[column]

        for column in positiveFeedbackdf:
            positiveFeedbackdf.loc[i,column] = insight.page_positive_feedback_by_type[column]

        for column in pageFansOnlinedf:
            pageFansOnlinedf.loc[i,column] = insight.page_fans_online[column]

    # Page fans online
    i=0
    for column in pageFansOnlinedf:
        onlineFansaveragedf.loc[i] = pageFansOnlinedf[column].sum()//24
        i+=1

    # Reactions
    for reaction in reactions['data']:
        i = 0
        for day in reaction['values']:
            reactionsdf.loc[i,reaction['name']] = day['value']
            i+=1
    reactionsdf.columns = ['Like','Love','Wow','Haha','Sorry','Anger']

    #Fans by country
    fansByCountrydf = pd.DataFrame(columns=countryCodes[2][0].iloc[1:].tolist())
    for metric in fansbyCountry['data']:
        i = 0
        for day in metric['values']:
            for v,k in enumerate(day['value']):
                fansByCountrydf.loc[i,k] = day['value'][k]
            i+=1
    fansByCountrydf.columns = countryCodes[2][1].iloc[1:].tolist()
    fansByCountrydf = fansByCountrydf.dropna(axis=1)
    averageFansByCountrydf = pd.DataFrame(columns=list(fansByCountrydf))
    for column in fansByCountrydf:
        averageFansByCountrydf.loc[0,column] = fansByCountrydf[column].sum()//len(dates)

    # Unlikes
    for metric in unlikes['data']:
        i = 0
        for day in metric['values']:
            for v,k in enumerate(day['value']):
                pageUnlikesBySourcedf.loc[i,k] = day['value'][k]
            i+=1
    pageUnlikesBySourcedf = pageUnlikesBySourcedf.iloc[:,0:2]

    # Likes
    for metric in likes['data']:
        i = 0
        for day in metric['values']:
            for v,k in enumerate(day['value']):
                pageLikesBySourcedf.loc[i,k] = day['value'][k]
            i+=1
    pageLikesBySourcedf =  pageLikesBySourcedf.fillna(0)

    # Fans
    for metric in fans['data']:
        i = 0
        for day in metric['values']:
            fansDemographicsdf.loc[i,metric['name']] = day['value']
            i+=1
    for metric in fansByAge['data']:
        i = 0
        for day in metric['values']:
            for v,k in enumerate(day['value']):
                fansbyAgedf.loc[i,k] = day['value'][k]
            i+=1
    fansDemographicsdf = fansDemographicsdf.join(fansbyAgedf)
    averageFansDemographicsdf = pd.DataFrame(columns=list(fansDemographicsdf))
    for column in fansDemographicsdf:
        averageFansDemographicsdf.loc[0,column] = fansDemographicsdf[column].sum()//len(dates)
    ageRanges = ["13-17","18-24","25-34","35-44","45-54","55-64","65+"]
    for age in ageRanges:
        s = 0
        for column in averageFansDemographicsdf:
            if column[2:] == age:
                s+=averageFansDemographicsdf.loc[0,column]
        newdf.loc[0,age] = s
    newdf = newdf.join(averageFansDemographicsdf["page_fans"])
    averageFansDemographicsdf = newdf

    # Index as date
    dfarray = [pageFansOnlinedf,positiveFeedbackdf,negativeFeedbackdf,impressionsdf,pageConsumptionsdf,reactionsdf,fansByCountrydf,pageLikesBySourcedf,pageUnlikesBySourcedf,fansDemographicsdf]
    for df in dfarray:
        df.index = dates
    averageFansDemographicsdf.index = ['Average']
    averageFansByCountrydf.index = ['Average']

    # Fixing dataframes...
    positiveFeedbackdf = positiveFeedbackdf.drop(['answer','claim'],axis=1)
    negativeFeedbackdf = negativeFeedbackdf.drop(['report_spam_clicks','unlike_page_clicks','xbutton_clicks'],axis=1)
    averageFansByCountrydf = averageFansByCountrydf.transpose().sort_index().sort_values('Average', ascending=False)
    
    writer = pd.ExcelWriter('Datasets/'+since.split(' ')[2]+'/'+since.split(' ')[0]+'.xlsx')
    onlineFansaveragedf.to_excel(writer, sheet_name='pageFansOnline')
    positiveFeedbackdf.to_excel(writer, sheet_name='positiveFeedback')
    negativeFeedbackdf.to_excel(writer, sheet_name='negativeFeedback')
    impressionsdf.to_excel(writer, sheet_name='impressions')
    pageConsumptionsdf.to_excel(writer, sheet_name='pageConsumptions')
    reactionsdf.to_excel(writer, sheet_name='reactions')
    averageFansByCountrydf.to_excel(writer, sheet_name='fansByCountry')
    pageLikesBySourcedf.to_excel(writer, sheet_name='pageLikesBySource')
    pageUnlikesBySourcedf.to_excel(writer, sheet_name='pageUnlikesBySource')
    averageFansDemographicsdf.to_excel(writer, sheet_name='averageFansDemographics')
    # videosdf.to_excel(writer, sheet_name='videos')
    writer.save()
    print("Datasets generated.")


# Calling function to generate datasets for certain month

In [2]:
generateCSVs(token='EAACEdEose0cBAJhXEz7zDoLacc1rL6nZCjBkhL145bXqBqM5OvmXy7wPyCjxkketmRgNP8rcWUQ28ywEJqSgINYgDPl81vHAsI2uujhV3RGv23aoFQFcy5f1b4jFZCqBGHwvuntCRTd1cbn8TKnf5LPlV273giRXTVHroCd3RjUQJ5rSBtvyhlnK75cZC0ZD', since ='July 1 2017', until = 'August 1 2017')

Datasets generated.
         0      1      2      3       4       5       6       7       8  \
02  130433  68274  49426  48120   70048  123300  186415  239341  272116   
03  134114  78610  55714  46873   56594  100303  159091  226897  263816   
04   98531  51109  41349  55849   99359  156701  208450  250022  274176   
05   94192  49362  40602  55681  100041  159866  211473  251891  277711   
06   94338  49128  39594  54270   99703  160553  213192  251902  275078   
07   98524  53022  41232  54551   97816  157233  209425  249829  273195   
08  105489  52339  41494  54741   96924  157711  210055  250482  272924   
09  127599  66611  48996  47956   69281  120585  183148  237840  270559   
10  132812  77650  55288  46090   54334   91040  149836  212796  257857   
11   99483  51708  41222  55016   94864  151098  204497  247440  273689   
12   94629  49836  39860  53384   93934  153352  204490  246936  272044   
13   94813  48872  39550  52735   92780  152090  204819  247003  271176   
14   

# Video report

In [ ]:
def videoGenerateCSVs(token, since, until):
    # page video views organic
    # page video view autoplay
    # page video view click to play
    # page video complete views: autoplay, click to play
    page = fi.authenticate(token=token)
    graph = facebook.GraphAPI(access_token=token, version='2.7')
    
    videos = graph.get_object(id="tecdemonterrey/insights",metric='page_video_views_organic,page_video_views_autoplayed,page_video_views_click_to_play,page_video_complete_views_30s_autoplayed,page_video_complete_views_30s_click_to_play',until=until,since=since)
    videosdf = pd.DataFrame(columns=['page_video_views_organic','page_video_views_autoplayed','page_video_views_click_to_play','page_video_complete_views_30s_autoplayed','page_video_complete_views_30s_click_to_play'])
    for video in videos['data']:
        i = 0
        for day in video['values']:
            videosdf.loc[i,video['name']] = day['value']
            i+=1
    videosdf.index = [i for i in range(1,len(videosdf.index)+1)]
    print(videosdf)
    

videoGenerateCSVs(token='EAACEdEose0cBAEYtcVZAe6bGhWAz9qlrW6EfuGVRPXwbkcEI6EiDNTf5BU9kHtkCZCotZCrQrJYZBe1ZBCQJNGuvtnA3ew0r51XvZAh45qZBy2QMO4qRAVL74ZBVWWmS0EX4R98YynoQqhrUZBZCGUL3pIQLFScAvSMGGbT3lUxPpdeRi47ZAMvZAMvHZCsYaf4dQDX4ZD',since ='February 1 2017',until = 'February 29 2017')